In [141]:
import pandas as pd
import os

In [3]:
def get_all_councils():
    file_path = os.path.join('DataBase', 'db.xlsx')
    df = pd.read_excel(file_path, sheet_name="DS Hội đồng_DN", skiprows=2)
    df = df.dropna(ignore_index=True)
    df = df[df['Họ và tên'] != 'Họ và tên']
    df

    councils = {}
    for i in range(len(df)):
        member = df.iloc[i]
        name = member['Họ và tên']
        role = member['Nhiệm vụ'].replace('\xa0', '')
        unit = member['Đơn vị']
        id_council = f"HD{(i)//5 + 1}"
        if councils.get(id_council) is None:
            councils[id_council] = [{
                'name': name,
                'role': role,
                'unit': unit
            }]
        else:
            councils[id_council].append({
                'name': name,
                'role': role,
                'unit': unit
            })
    return councils

In [61]:
def find_student_by_council_and_group_id(council_id=None, group_id=None):
    students = []
    if group_id is None or council_id is None:
        return students
    file_path = os.path.join('DataBase', 'db.xlsx')
    df2 = pd.read_excel(file_path, sheet_name="DS SV các Hội đồng phân PB", skiprows=1)
    df2 = df2.drop(columns = ['Unnamed: 0'])
    df2.loc[1, 'Unnamed: 11'] = df2.loc[0, 'Unnamed: 11']
    df2.loc[1, 'Unnamed: 12'] = df2.loc[0, 'Unnamed: 12']
    df2.loc[1, 'Unnamed: 3'] = 'Tên'
    df2.columns = df2.iloc[1]
    df2 = df2.dropna(how='all', ignore_index=True)
    index_council = find_index_hd_in_excel(df2, council_id)
    index_council_next = find_index_hd_in_excel(df2, council_id + 1)
    if index_council_next == -1:
        index_council_next = len(df2)
    df2 = df2.iloc[index_council+2:index_council_next]
    df2 = df2.reset_index(drop=True)
    for i in range(len(df2)):
        df2 = df2.reset_index(drop=True) 
        if df2.loc[i, 'Nhóm'] == group_id:
            students.append(df2.loc[i, 'Mã sinh viên'])
    print("Students", students)
    return students

def find_index_hd_in_excel(df2, id_hd):
    for i in range(len(df2)):
        if df2.loc[i, 'Mã sinh viên'] == f'Hội đồng {id_hd}':
            return i
    return -1

In [56]:
def get_students_by_project_name(project_name):
    file_path = os.path.join('DataBase', 'db.xlsx')
    df = pd.read_excel(file_path, sheet_name="Danh sách các đồ án ", skiprows=12)
    df = df.ffill()

    students = []
    col = 'Tên đề tài đồ án/ khóa luận tốt nghiệp'
    df_students = df[df[col].str.contains(project_name, case=False, regex=True)]
    
    first_students = list(df_students['Họ và tên'])
    second_students = list(df_students['Unnamed: 3'])
    fullname_students = [f"{first} {second}" for first, second in zip(first_students, second_students)]
    msv_students = list(df_students['Mã sinh viên'])
    day_of_birth_students = list(df_students['Năm sinh'])
    class_students = list(df_students['Lớp'])
    for i in range(len(fullname_students)):
        students.append({
            'fullname': fullname_students[i],
            'msv': msv_students[i],
            'day_of_birth': day_of_birth_students[i],
            'class': class_students[i]
        })
    return students

In [63]:
print('A', find_student_by_council_and_group_id(5, 1))

Students ['B20DCDT028', 'B20DCDT104', 'B20DCDT187', 'B20DCDT215', 'B20DCDT005']
A ['B20DCDT028', 'B20DCDT104', 'B20DCDT187', 'B20DCDT215', 'B20DCDT005']


In [57]:
# file_path = os.path.join('DataBase', 'db.xlsx')
# df2 = pd.read_excel(file_path, sheet_name="DS SV các Hội đồng phân PB", skiprows=1)
# df2 = df2.drop(columns = ['Unnamed: 0'])
# df2.loc[1, 'Unnamed: 11'] = df2.loc[0, 'Unnamed: 11']
# df2.loc[1, 'Unnamed: 12'] = df2.loc[0, 'Unnamed: 12']
# df2.loc[1, 'Unnamed: 3'] = 'Tên'
# df2.columns = df2.iloc[1]
# df2 = df2.dropna(how='all', ignore_index=True)
# councils = get_all_councils()
# id_council = 'HD1'
# students = {
# }
# for i in range(len(df2)):
#     msv = df2.loc[i, 'Mã sinh viên']
#     if 'Hội đồng' in msv:
#         tmp = msv.split(' ')
#         id_council = f"HD{tmp[-1]}"
#         continue
#     if msv == 'Mã sinh viên':
#         continue
#     name = df2.loc[i, 'Họ và tên'] + ' ' + df2.loc[i, 'Tên']
#     day_of_birth = df2.loc[i, 'Năm sinh']
#     class_name = df2.loc[i, 'Lớp']
#     instructor = df2.loc[i, 'Giáo viên hướng dẫn']
#     subject = df2.loc[i, 'Bộ môn ']
#     project = df2.loc[i, 'Tên đề tài đồ án/ khóa luận tốt nghiệp']
#     project_type = df2.loc[i, 'Loại đồ án']
#     group = id_council + ' - ' + str(df2.loc[i, 'Nhóm']) if not pd.isnull(df2.loc[i, 'Nhóm']) else ''
#     opponent = df2.loc[i, 'Phản biện  (Khoa)'] if not pd.isna(df2.loc[i, 'Phản biện  (Khoa)']) else ''
#     council = councils[id_council]
#     students[msv] = {
#         'name': name,
#         'day_of_birth': day_of_birth,
#         'class_name': class_name,
#         'instructor': instructor,
#         'subject': subject,
#         'project': project,
#         'project_type': project_type,
#         'group': group,
#         'opponent': opponent,
#         'council': council,
#         'msv': msv
#     }
#     # Get students from final sheet
# new_students = students.copy()
# for msv in students.keys():
#     student = students[msv]
#     if student['project'] == 'None':
#         continue
#     project_name = student['project']
#     students_same_project = get_students_by_project_name(project_name)
#     for std in students_same_project:
#         msv_std = std['msv']
#         if msv_std not in students.keys():
#             fullname = std['fullname']
#             day_of_birth = std['day_of_birth']
#             class_name = std['class']

#             new_students[msv_std] = students[msv].copy()
#             new_students[msv_std]['msv'] = msv_std
#             new_students[msv_std]['name'] = fullname
#             new_students[msv_std]['day_of_birth'] = day_of_birth
#             new_students[msv_std]['class_name'] = class_name


C:\Users\xin chao\AppData\Local\Temp\ipykernel_14812\4292890001.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_students = df[df[col].str.contains(project_name, case=False, regex=True)]
C:\Users\xin chao\AppData\Local\Temp\ipykernel_14812\4292890001.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_students = df[df[col].str.contains(project_name, case=False, regex=True)]


In [ ]:
# import json
# with open('students.json', 'w', encoding='utf-8') as f:
#     json.dump(new_students, f, ensure_ascii=False, indent=4)

    